In [160]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import re
import os
from collections import defaultdict

In [161]:
def preprocess(img):
    img = cv.GaussianBlur(img, (3, 3), 0)
    img = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(img)
    return img

In [162]:
def keypoints(I1, I2):
    sift = cv.SIFT_create()  # or cv.AKAZE_create() / cv.ORB_create() for speed
    kp1, des1 = sift.detectAndCompute(I1, None)
    kp2, des2 = sift.detectAndCompute(I2, None)
    return (kp1, des1), (kp2, des2)

In [163]:
def match(des1, des2):
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:  # Lowe's ratio test
            good.append(m)

    # print(f"Found {len(good)} good matches out of {len(matches)} total")
    return good

In [164]:
def ransac(kp1, kp2, good):
        pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in good])

        M, inliers = cv.estimateAffinePartial2D(
            pts2, pts1,
            method=cv.RANSAC,
            ransacReprojThreshold=3.0,
            maxIters=2000,
            confidence=0.99
        )

        # print("Affine matrix:\n", M)
        return M, inliers

In [166]:
def register(img1_full, img2_full):

    img1_gray = cv.cvtColor(img1_full, cv.COLOR_BGR2GRAY)
    img2_gray = cv.cvtColor(img2_full, cv.COLOR_BGR2GRAY)

    I1, I2 = preprocess(img1_gray), preprocess(img2_gray)

    # --- 1. Detect and describe keypoints ---
    (kp1, des1), (kp2, des2) = keypoints(I1, I2)

    # --- 2. Match descriptors with ratio test ---
    if kp1 is not None and kp2 is not None and des1 is not None and des2 is not None:
        good = match(des1, des2)

        # --- 3. Estimate affine transform using RANSAC ---
        if len(good) >= 3:  # need at least 3 points for affine
            M, inliers = ransac(kp1, kp2, good)

            # If you need to enforce only translation, you can extract the translation components
            # from the estimated matrix and create a new purely translational matrix.
            # The translation components are in the last column of the affine matrix.
            tx = M[0, 2]
            ty = M[1, 2]

            translation_matrix = np.array([[1, 0, tx],
                                           [0, 1, ty]], dtype=np.float32)

            print("\nPurely Translational Matrix:")
            print(translation_matrix)

            rows, cols, _ = img2_full.shape
            int_tx = int(round(tx))
            int_ty = int(round(ty))
            return (int_tx, int_ty)

        else:
            print("Not enough good matches for reliable registration.")
    else:
        print("Not enough descriptors for reliable registration.")

In [167]:
def pipeline(fname1, fname2, dimensions):

    fname1_path = "slu_data/" + fname1
    fname2_path = "slu_data/" + fname2
    img1 = cv.imread(fname1_path)    # reference (earlier)
    img2 = cv.imread(fname2_path)    # moving (later)
    dimensions[fname1] = img1.shape
    dimensions[fname2] = img2.shape
    result = register(img1, img2)

    if result is not None:
        return result
    else:
        return False

In [168]:
def group_and_order_filenames(filenames, maxTP, maxLevel):
    grouped_files = defaultdict(lambda: defaultdict(lambda: [None] * maxLevel))
    pattern = r'^(?P<plant>[^_]+)_(?P<tube>\d+)_(?P<level>\d+)_(?P<date>\d{4}-\d{2}-\d{2})_TP(?P<timepoint>\d+)\.png$'
    #plant_tube_depth_yyyy-mm-dd_TP#

    for fname in filenames:

        match = re.match(pattern, fname)
        if match:
            plant = match.group('plant')
            tube = int(match.group('tube'))
            level = int(match.group('level'))
            date = match.group('date')
            timepoint = int(match.group('timepoint'))
            if 1 <= level <= maxLevel:
                grouped_files[tube][timepoint - 1][level - 1] = fname

    return grouped_files

In [169]:
def accumulate(filename, translation, translations, references, all):
    if filename not in references:
        return translation

    ref = references[filename]
    if ref not in translations or ref not in all:
        ref_translation = (0,0)
        return tuple(map(sum, zip(translation, ref_translation)))
    else:
        ref_translation = translations[ref]
        all.remove(ref)
        return tuple(map(sum, zip(translation, accumulate(ref, ref_translation, translations, references, all))))

In [170]:
imgfilelist = [f for f in os.listdir("slu_data") if f.endswith(".png")]
print(f"Found {len(imgfilelist)} image files")

imgfilegroups = group_and_order_filenames(imgfilelist, 12, 7)
print(f"Found {len(imgfilegroups)} image groups")

Found 21 image files
Found 21 image groups


In [171]:
for tube, timepoints in imgfilegroups.items():
    for tp, d_files in timepoints.items():

        non_none_files = list(filter(None, d_files))
        my_iterator = iter(non_none_files)
        translations = {}
        references = {}
        dimensions = {}
        try:

            current_item = next(my_iterator)
            next_item = next(my_iterator)
            translations[current_item] = (0,0)
            while current_item and next_item:

                translation = pipeline(current_item, next_item, dimensions)
                if translation:
                    translations[next_item] = translation
                    references[next_item] = current_item

                inner_iterator = iter(non_none_files)
                try:
                    new_item = next(inner_iterator)
                    while not translation and new_item:
                        if new_item == next_item or new_item == current_item:
                            new_item = next(inner_iterator)
                            continue

                        translation = pipeline(new_item, next_item, dimensions)
                        if translation:
                            translations[next_item] = translation
                            references[next_item] = new_item
                        new_item = next(inner_iterator)

                except StopIteration:
                    current_item = next_item
                    next_item = next(my_iterator)
                    continue

                current_item = next_item
                next_item = next(my_iterator)

        except StopIteration:

            global_translations = {}
            for f_accumulate, t_accumulate in translations.items():
                try:
                    all_compared = non_none_files.copy()
                    global_t = accumulate(f_accumulate, t_accumulate, translations, references, all_compared)
                    global_translations[f_accumulate] = global_t
                except RecursionError:
                    print("f_accumulate: " + f_accumulate)

            global_x = []
            global_y = []
            for key, value in global_translations.items():
                global_x.extend([value[0], value[0] + dimensions[key][1]])
                global_y.extend([value[1], value[1] + dimensions[key][0]])

            min_x = min(global_x)
            min_y = min(global_y)
            max_x = max(global_x)
            max_y = max(global_y)
            final_cols  = max_x - min_x
            final_rows = max_y - min_y
            offset_x = -min_x
            offset_y = -min_y

            stitched_img = np.zeros((final_rows, final_cols, 3), dtype=img.dtype)
            for f, t in global_translations.items():
                f_no_ext, f_ext = os.path.splitext(f)
                p = "slu_data/" + f
                img = cv.imread(p)

                c_start = t[0] + offset_x
                r_start = t[1] + offset_y
                # new_img = np.zeros((final_rows, final_cols, 3), dtype=img.dtype)

                try:
                    # new_img[r_start:r_start + img.shape[0], c_start:c_start + img.shape[1]] = img
                    stitched_img[r_start:r_start + img.shape[0], c_start:c_start + img.shape[1]] = img
                except ValueError:
                    print("f: " + f)

                # cv.imwrite("slu_data/" + f_no_ext + "_sift_depths.png", new_img)
            cv.imwrite("slu_data/kura_" + str(tube) + "_TP" + str(tp + 1) + "_sift_depths.png", stitched_img)

            continue


Purely Translational Matrix:
[[  1.          0.        773.06494  ]
 [  0.          1.          6.6860194]]

Purely Translational Matrix:
[[  1.         0.       799.33765 ]
 [  0.         1.         4.508773]]

Purely Translational Matrix:
[[  1.          0.        801.3615   ]
 [  0.          1.          5.7434916]]

Purely Translational Matrix:
[[  1.         0.       800.76825 ]
 [  0.         1.         5.448008]]

Purely Translational Matrix:
[[  1.         0.       801.3292  ]
 [  0.         1.         6.064692]]

Purely Translational Matrix:
[[  1.          0.        806.36676  ]
 [  0.          1.          4.5280976]]

Purely Translational Matrix:
[[  1.          0.        809.51025  ]
 [  0.          1.          1.4867431]]

Purely Translational Matrix:
[[  1.          0.        795.17194  ]
 [  0.          1.          3.2237055]]

Purely Translational Matrix:
[[  1.          0.        798.9673   ]
 [  0.          1.          3.3665955]]

Purely Translational Matrix:
[[  1. 